USER TO ITEM RECOMMENDATION SYSTEM

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
vectorizer = TfidfVectorizer()
item_to_item_model_data = pd.read_csv("data/item_to_item_model_data.csv", index_col=0)
item_to_item_model_data = item_to_item_model_data.reset_index(drop=True)

FileNotFoundError: [Errno 2] No such file or directory: 'data/item_to_item_model_data.csv'

In [ ]:
# Create similarity matrix based on popular genres
popular_genres_matrix = vectorizer.fit_transform(item_to_item_model_data['popular_genres'])
popular_genres_similarity_matrix = cosine_similarity(popular_genres_matrix, popular_genres_matrix)
popular_genres_similarity_matrix = pd.DataFrame(popular_genres_similarity_matrix)
#popular_genres_similarity_matrix.to_parquet('popular_genres_similarity_matrix.parquet', index=False)

# Create similarity matrix based on common genres
common_genres_matrix = vectorizer.fit_transform(item_to_item_model_data['common_genres'])
common_genres_similarity_matrix = cosine_similarity(common_genres_matrix, common_genres_matrix)
#common_genres_similarity_matrix = pd.DataFrame(common_genres_similarity_matrix)
#common_genres_similarity_matrix.to_parquet('common_genres_similarity_matrix.parquet', index=False)

# Create similarity matrix based on unpopular genres
unpopular_genres_matrix = vectorizer.fit_transform(item_to_item_model_data['unpopular_genres'])
unpopular_genres_similarity_matrix = cosine_similarity(unpopular_genres_matrix, unpopular_genres_matrix)
#unpopular_genres_similarity_matrix = pd.DataFrame(unpopular_genres_similarity_matrix)
#unpopular_genres_similarity_matrix.to_parquet('unpopular_genres_similarity_matrix.parquet', index=False)

In [ ]:
popular_genres_similarity_matrix = pd.DataFrame(popular_genres_similarity_matrix)

In [ ]:
def recommend_similar_games(title):

    # Encuentra el índice del juego ingresado
    game_index = item_to_item_model_data[item_to_item_model_data['title'] == title].index[0]

    # Combine matrices
    popular = popular_genres_similarity_matrix[game_index]
    common = common_genres_similarity_matrix[game_index]
    unpopular = unpopular_genres_similarity_matrix[game_index]
    combined_similarity_matrix = popular + common + unpopular

    # Ordena los juegos según su similitud (excluyendo el juego de entrada) y toma los 5 más similares
    similar_game_indices = combined_similarity_matrix.argsort()[::-1][0:6]

    # Obtiene los nombres de los juegos similares y sus puntajes de similitud
    similar_game_scores = combined_similarity_matrix[similar_game_indices]
    similar_game_titles = item_to_item_model_data.iloc[similar_game_indices]['title'].tolist()

    popular_game_scores = combined_similarity_matrix[similar_game_indices]
    popular_game_tags = item_to_item_model_data.iloc[similar_game_indices]['popular_genres'].tolist()
    popular_game_tags = item_to_item_model_data.iloc[similar_game_indices]['popular_genres'].tolist()

    common_game_scores = combined_similarity_matrix[similar_game_indices]
    common_game_tags = item_to_item_model_data.iloc[similar_game_indices]['common_genres'].tolist()

    unpopular_game_scores = combined_similarity_matrix[similar_game_indices]
    unpopular_game_tags = item_to_item_model_data.iloc[similar_game_indices]['unpopular_genres'].tolist()

    # Crea un DataFrame con los títulos y los puntajes de similitud
    similar_games_df = pd.DataFrame({'combined_score': similar_game_scores,
                                     'title': similar_game_titles,
                                     'pop_score': popular_game_scores,
                                     "pop_tags": popular_game_tags,
                                     'com_score': common_game_scores,
                                     "com_tags": common_game_tags,
                                     'unpop_score': unpopular_game_scores,
                                     "unpop_tags": unpopular_game_tags})

    return similar_games_df

similar_games = recommend_similar_games("TEKKEN 7")
similar_games

,combined_score,title,pop_score,pop_tags,com_score,com_tags,unpop_score,unpop_tags
9990,3.000000,TEKKEN 7,3.000000,"['Action', 'Singleplayer']",3.000000,"['Fighting', 'Multiplayer', 'Arcade', 'Competi...",3.000000,['e-sports']
25432,2.494381,Arcana Heart 3 LOVE MAX!!!!!,2.494381,"['Action', 'Singleplayer']",2.494381,"['Anime', 'Fighting', '2D Fighter', 'Local Mul...",2.494381,['e-sports']
6605,2.460239,UNDER NIGHT IN-BIRTH Exe:Late,2.460239,"['Action', 'Singleplayer']",2.460239,"['Anime', '2D Fighter', 'Fighting', 'Great Sou...",2.460239,['e-sports']
9142,2.387384,CHAOS CODE -NEW SIGN OF CATASTROPHE-,2.387384,"['Action', 'Singleplayer']",2.387384,"['2D Fighter', 'Fighting', 'Anime', 'Cute', 'D...",2.387384,['e-sports']
26161,2.222832,Battle Fantasia -Revised Edition-,2.222832,"['Action', 'Singleplayer']",2.222832,"['Fighting', '2D Fighter', 'Anime', 'Local Mul...",2.222832,"['2.5D', 'e-sports']"
31845,2.195014,Quake III Arena,2.195014,"['Action', 'Singleplayer']",2.195014,"['FPS', 'Classic', 'Arena Shooter', 'Multiplay...",2.195014,['e-sports']
